In [72]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

In [73]:
churnData = pd.read_csv('./files_for_lab/Customer-Churn.csv')

In [74]:
churnData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No,Yes,No,No,No,No,Month-to-month,29.85,29.85,No
1,Male,0,No,No,34,Yes,Yes,No,Yes,No,No,No,One year,56.95,1889.5,No
2,Male,0,No,No,2,Yes,Yes,Yes,No,No,No,No,Month-to-month,53.85,108.15,Yes
3,Male,0,No,No,45,No,Yes,No,Yes,Yes,No,No,One year,42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,No,No,No,No,No,Month-to-month,70.70,151.65,Yes


In [75]:
churnData.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [76]:
churnData['TotalCharges'] = pd.to_numeric(churnData['TotalCharges'], errors='coerce')

In [77]:
churnData.isna().sum()/len(churnData)

gender              0.000000
SeniorCitizen       0.000000
Partner             0.000000
Dependents          0.000000
tenure              0.000000
PhoneService        0.000000
OnlineSecurity      0.000000
OnlineBackup        0.000000
DeviceProtection    0.000000
TechSupport         0.000000
StreamingTV         0.000000
StreamingMovies     0.000000
Contract            0.000000
MonthlyCharges      0.000000
TotalCharges        0.001562
Churn               0.000000
dtype: float64

In [78]:
churnData = churnData.fillna(np.mean(churnData['TotalCharges']))

Use the following features: tenure, SeniorCitizen, MonthlyCharges and TotalCharges:

Scale the features either by using normalizer or a standard scaler.

Split the data into a training set and a test set.

Fit a logistic regression model on the training data.

Check the accuracy on the test data.


In [79]:
features = churnData[['tenure', 'SeniorCitizen', 'MonthlyCharges', 'TotalCharges']]

In [80]:
from sklearn.preprocessing import StandardScaler
standardized = pd.DataFrame(StandardScaler().fit_transform(features), columns=features.columns)

In [81]:
standardized

,tenure,SeniorCitizen,MonthlyCharges,TotalCharges
0,-1.277445,-0.439916,-1.160323,-0.994971
1,0.066327,-0.439916,-0.259629,-0.173876
2,-1.236724,-0.439916,-0.362660,-0.960399
3,0.514251,-0.439916,-0.746535,-0.195400
4,-1.236724,-0.439916,0.197365,-0.941193
...,...,...,...,...
7038,-0.340876,-0.439916,0.665992,-0.129281
7039,1.613701,-0.439916,1.277533,2.242808
7040,-0.870241,-0.439916,-1.168632,-0.855182
7041,-1.155283,2.273159,0.320338,-0.872777


In [82]:
y = churnData['Churn']

In [83]:
y = y.apply(lambda x: 1 if x=='Yes' else 0)

In [84]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(standardized, y, random_state=0)

In [85]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression(random_state=0).fit(X_train, y_train)

In [86]:
pred = LR.predict(X_test)

In [87]:
LR.score(X_test, y_test)

0.7830777967064169

In [97]:
print("precision: ",precision_score(y_test,pred))

precision:  0.6180758017492711


# Imbalance

In [88]:
y.value_counts()

0    5174
1    1869
Name: Churn, dtype: int64

In [89]:
from sklearn.utils import resample

In [90]:
train = pd.concat([X_train, y_train], axis=1)

In [91]:
category_0 = train[train['Churn'] == 0]
category_1 = train[train['Churn'] == 1]

## Oversampling

In [92]:
category_1_oversampled = resample(category_1, 
                                  replace=True, 
                                  n_samples = len(category_0))

In [100]:
oversampled = pd.concat([category_1_oversampled, category_0], axis=0)

In [94]:
oversampled['Churn'].value_counts()

0    3876
1    3876
Name: Churn, dtype: int64

In [95]:
y_train_over = oversampled['Churn'].copy()
X_train_over = oversampled.drop('Churn',axis = 1).copy()

In [98]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

LR_over = LogisticRegression(max_iter=1000)
LR_over.fit(X_train_over, y_train_over)
pred_over = LR_over.predict(X_test)



print("r2: ",LR_over.score(X_test, y_test))
print("precision: ",precision_score(y_test,pred_over))


r2:  0.720045428733674
precision:  0.4782608695652174


# Undersampling

In [103]:
category_0_undersampled = resample(category_0, 
                                  replace=True, 
                                  n_samples = len(category_1))
undersampled = pd.concat([category_0_undersampled, category_1], axis=0)
y_train_under = undersampled['Churn'].copy()
X_train_under = undersampled.drop('Churn',axis = 1).copy()

In [104]:
LR_under = LogisticRegression(max_iter=1000)
LR_under.fit(X_train_under, y_train_under)
pred_under = LR_under.predict(X_test)



print("r2: ",LR_under.score(X_test, y_test))
print("precision: ",precision_score(y_test,pred_under))

r2:  0.7183418512208972
precision:  0.47639484978540775


## SMOTE

In [105]:
from imblearn.over_sampling import SMOTE

In [120]:
sm = SMOTE(random_state=100)
X_SMOTE,y_SMOTE = sm.fit_resample(X_train,y_train)


In [121]:
X_SMOTE.shape

(7752, 4)

In [122]:
y_SMOTE.value_counts()

0    3876
1    3876
Name: Churn, dtype: int64

In [123]:
LR_smote = LogisticRegression(max_iter=1000)
LR_smote.fit(X_SMOTE,y_SMOTE)
pred_smote = LR_smote.predict(X_test)



print("r2: ",LR_smote.score(X_test, y_test))
print("precision: ",precision_score(y_test,pred_smote))

r2:  0.717206132879046
precision:  0.4752475247524752
